⚠️ NOTE:
This notebook is for research and validation only.
Production logic lives in the `src/` directory.

In [1]:
# Project path
import sys
from pathlib import Path

PROJECT_ROOT = Path(r"C:\Users\shubh\crypto-market-opportunity-engine")
sys.path.insert(0, str(PROJECT_ROOT))

import pandas as pd
import numpy as np
import src.config as config


In [2]:
# load signal data
DATA_SIGNALS = r"C:\Users\shubh\crypto-market-opportunity-engine\data\processed\BTCUSDT_5m_signals.parquet"

df = pd.read_parquet(DATA_SIGNALS)

print("Shape:", df.shape)
df[["close", "prob_up", "signal"]].tail()


Shape: (105092, 28)


,close,prob_up,signal
open_time,,,
2025-12-31 23:35:00,87652.47,0.492735,HOLD
2025-12-31 23:40:00,87685.31,0.494017,HOLD
2025-12-31 23:45:00,87690.61,0.492686,HOLD
2025-12-31 23:50:00,87641.14,0.489541,HOLD
2025-12-31 23:55:00,87648.22,0.488097,HOLD


In [3]:
# alert rules
MIN_CONFIDENCE = 0.60

df["prev_signal"] = df["signal"].shift(1)

df["alert"] = (
    (df["signal"] != df["prev_signal"]) &
    (df["signal"].isin(["BUY", "SELL"])) &
    (
        ((df["signal"] == "BUY") & (df["prob_up"] >= MIN_CONFIDENCE)) |
        ((df["signal"] == "SELL") & (df["prob_up"] <= 1 - MIN_CONFIDENCE))
    )
)


In [4]:
# alert message builder
def build_alert(row):
    return (
        f"🚨 {row['signal']} SIGNAL\n"
        f"Price: {row['close']:.2f}\n"
        f"Confidence: {row['prob_up']:.2%}\n"
        f"Time: {row.name}"
    )

alerts = df[df["alert"]].copy()
alerts["message"] = alerts.apply(build_alert, axis=1)


In [5]:
# test_alerts console
for msg in alerts["message"].head(5):
    print("=" * 40)
    print(msg)


🚨 SELL SIGNAL
Price: 101020.18
Confidence: 39.79%
Time: 2025-01-06 14:50:00
🚨 BUY SIGNAL
Price: 95639.99
Confidence: 60.36%
Time: 2025-01-08 06:35:00
🚨 BUY SIGNAL
Price: 94641.45
Confidence: 62.94%
Time: 2025-01-08 11:35:00
🚨 BUY SIGNAL
Price: 94079.21
Confidence: 64.89%
Time: 2025-01-09 02:10:00
🚨 BUY SIGNAL
Price: 92876.00
Confidence: 64.07%
Time: 2025-01-10 13:35:00


In [6]:
# add sanity check
print("Total alerts:", len(alerts))
print(alerts["signal"].value_counts())


Total alerts: 47
signal
BUY     37
SELL    10
Name: count, dtype: int64
